In [18]:
import iris
import numpy

## Figure out how weighting works

#### Iris area weighting

In [44]:
raw_data_file = '/g/data/r87/dbi599/drstree/CMIP5/GCM/CCCMA/CanESM2/historical/yr/ocean/thetao/r1i1p1/dedrifted/thetao_Oyr_CanESM2_historical_r1i1p1_194101-195012.nc'

In [66]:
raw_data_cube = iris.load_cube(raw_data_file, 'sea_water_potential_temperature')
raw_data_cube = raw_data_cube

In [67]:
print raw_data_cube

sea_water_potential_temperature / (K) (time: 10; depth: 40; latitude: 192; longitude: 256)
     Dimension coordinates:
          time                             x          -             -               -
          depth                            -          x             -               -
          latitude                         -          -             x               -
          longitude                        -          -             -               x
     Attributes:
          CCCma_data_licence: 1) GRANT OF LICENCE - The Government of Canada (Environment Canada) is...
          CCCma_parent_runid: IGA
          CCCma_runid: IGM
          Conventions: CF-1.5
          associated_files: baseURL: http://cmip-pcmdi.llnl.gov/CMIP5/dataLocation gridspecFile: gridspec_ocean_fx_CanESM2_historical_r0i0p0.nc...
          branch_time: 171915.0
          branch_time_YMDH: 2321:01:01:00
          cmor_version: 2.5.4
          contact: cccma_info@ec.gc.ca
          creation_date: 2011-03-28

In [68]:
print raw_data_cube.data.mean()

280.946605958


In [69]:
area_weights = iris.analysis.cartography.area_weights(raw_data_cube, normalize=False)

In [70]:
print area_weights.shape
print area_weights.mean()

(10, 40, 192, 256)
10365791879.2


In [71]:
weighted_spatial_mean_cube = raw_data_cube.collapsed(['longitude', 'latitude'], iris.analysis.MEAN, weights=area_weights)
spatial_mean_cube = raw_data_cube.collapsed(['longitude', 'latitude'], iris.analysis.MEAN)

In [72]:
print weighted_spatial_mean_cube.data.mean()
print spatial_mean_cube.data.mean()

281.891596085
279.998735386


#### My vertical weighting

In [65]:
def broadcast_array(array, axis_index, shape):
    """Broadcast an array to a target shape.
    
    Args:
      array (numpy.ndarray): One dimensional array
      axis_index (int or tuple): Postion in the target shape that the 
        axis/axes of the array corresponds to
          e.g. if array corresponds to (lat, lon) in (time, depth lat, lon)
          then axis_index = [2, 3]
          e.g. if array corresponds to (lat) in (time, depth lat, lon)
          then axis_index = 2
      shape (tuple): shape to broadcast to
      
    For a one dimensional array, make start_axis_index = end_axis_index
    
    """

    if type(axis_index) in [float, int]:
        start_axis_index = end_axis_index = axis_index
    else:
        assert len(axis_index) == 2
        start_axis_index, end_axis_index = axis_index
    
    dim = start_axis_index - 1
    while dim >= 0:
        array = array[numpy.newaxis, ...]
        array = numpy.repeat(array, shape[dim], axis=0)
        dim = dim - 1
    
    dim = end_axis_index + 1
    while dim < len(shape):    
        array = array[..., numpy.newaxis]
        array = numpy.repeat(array, shape[dim], axis=-1)
        dim = dim + 1

    return array


def calc_vertical_weights_1D(depth_coord, coord_names, data_shape):
    """Calculate vertical weights for a 1D depth axis with units = m.
    Args:
      depth_coord (iris.coords.DimCoord): One-dimensional depth coordinate
      coord_names (list): Names of each data coordinate
      data_shape (tuple): Shape of data
    Returns:
      iris.cube: Array of weights with shape matching data_shape
  
    """

    assert depth_coord.units == 'm'

    # Calculate weights
    if not depth_coord.has_bounds():
        depth_coord.guess_bounds()
    level_bounds = depth_coord.bounds
    level_diffs = numpy.apply_along_axis(lambda x: x[1] - x[0], 1, level_bounds)

    #guess_bounds can produce negative bound at surface
    if level_bounds[0][0] < 0.0:
        level_diffs[0] = level_diffs[0] + level_bounds[0][0]

    # Broadcast to size of data
    depth_index = coord_names.index('depth')
    level_diffs = broadcast_array(level_diffs, depth_index, data_shape)

    return level_diffs

In [73]:
depth_axis = raw_data_cube.coord('depth')
coord_names = [coord.name() for coord in raw_data_cube.dim_coords]
vertical_weights = calc_vertical_weights_1D(depth_axis, coord_names, raw_data_cube.shape)

In [80]:
print vertical_weights.shape
print vertical_weights[0,:,50, 40]

(10, 40, 192, 256)
[  10.           10.           10.19999886   10.19999886   10.40000153
   10.59999847   10.79999924   11.           11.40000153   11.79999542
   12.40000153   13.40000153   14.59999084   16.20001221   18.3999939
   21.3999939    25.20001221   30.19998169   36.80001831   45.3999939
   56.3999939    70.19998169   87.00003052  107.39996338  130.79998779
  157.20001221  185.39996338  214.59997559  243.40002441  270.59997559
  295.          316.39990234  334.40014648  349.19995117  361.
  370.39990234  377.60009766  383.19970703  387.40039062  390.59960938]


In [81]:
vertical_mean_cube = raw_data_cube.collapsed(['depth'], iris.analysis.MEAN, weights=vertical_weights.astype(numpy.float32))

In [85]:
print vertical_mean_cube.data.mean()

276.432410842


## Case 1: Use depth file


In [86]:
data_file = '/g/data/r87/dbi599/drstree/CMIP5/GCM/CCCMA/CanESM2/historical/yr/ocean/thetao-maps/r1i1p1/thetao-maps_Oyr_CanESM2_historical_r1i1p1_all.nc'
depth_file = '/g/data/ua6/drstree/CMIP5/GCM/CCCMA/CanESM2/historical/fx/ocean/deptho/r0i0p0/deptho_fx_CanESM2_historical_r0i0p0.nc'

In [87]:
data_cube = iris.load_cube(data_file, 'vertical mean argo sea water potential temperature')

In [90]:
print data_cube.data.mean()

5.84016370904e-08


In [91]:
depth_cube = iris.load_cube(depth_file)

In [92]:
print depth_cube

sea_floor_depth_below_geoid / (m)   (latitude: 192; longitude: 256)
     Dimension coordinates:
          latitude                           x               -
          longitude                          -               x
     Attributes:
          CCCma_data_licence: 1) GRANT OF LICENCE - The Government of Canada (Environment Canada) is...
          CCCma_parent_runid: IGA
          CCCma_runid: IGY
          Conventions: CF-1.4
          associated_files: baseURL: http://cmip-pcmdi.llnl.gov/CMIP5/dataLocation gridspecFile: gridspec_ocean_fx_CanESM2_historical_r0i0p0.nc...
          branch_time: 171915.0
          branch_time_YMDH: 2321:01:01:00
          cmor_version: 2.5.4
          comment: Ocean bathymetry.   Reported here is the sea floor depth for present day....
          contact: cccma_info@ec.gc.ca
          creation_date: 2011-05-04T17:20:05Z
          experiment: historical
          experiment_id: historical
          forcing: GHG,Oz,SA,BC,OC,LU,Sl,Vl (GHG includes CO2,CH4

In [93]:
depth_array = depth_cube.data

depth_array = depth_array[numpy.newaxis, ...]
depth_array = numpy.repeat(depth_array, 156, axis=0)

In [94]:
print depth_array.shape
print data_cube.data.shape
print type(depth_array)

(156, 192, 256)
(156, 192, 256)
<class 'numpy.ma.core.MaskedArray'>


In [95]:
zonal_mean_cube = data_cube.collapsed(['longitude'], iris.analysis.MEAN, weights=depth_array) #weights=vertical_weights.astype(numpy.float32))

In [96]:
print zonal_mean_cube

vertical mean argo sea water potential temperature / (K) (time: 156; latitude: 192)
     Dimension coordinates:
          time                                                x              -
          latitude                                            -              x
     Scalar coordinates:
          longitude: 180.0 degrees, bound=(0.0, 360.0) degrees
     Attributes:
          CCCma_data_licence: 1) GRANT OF LICENCE - The Government of Canada (Environment Canada) is...
          CCCma_parent_runid: IGA
          CCCma_runid: IGM
          Conventions: CF-1.5
          associated_files: baseURL: http://cmip-pcmdi.llnl.gov/CMIP5/dataLocation gridspecFile: gridspec_ocean_fx_CanESM2_historical_r0i0p0.nc...
          branch_time: 171915.0
          branch_time_YMDH: 2321:01:01:00
          cmor_version: 2.5.4
          contact: cccma_info@ec.gc.ca
          drift_removal: Masked 0 of 1966080 points because cubic fit was poor
          experiment: historical
          experiment_id: his

### Validate result

In [98]:
print zonal_mean_cube.data[0,:].mean()

-0.019928713723


In [32]:
test_result = zonal_mean_cube.data[0,60]

In [33]:
print test_result

0.00591959036936


### Easier data...

In [101]:
print raw_data_cube[:,0,:,:].shape

easy_depth_array = depth_cube.data
easy_depth_array = easy_depth_array[numpy.newaxis, ...]
easy_depth_array = numpy.repeat(easy_depth_array, 10, axis=0)

print easy_depth_array.shape

#easy_depth_array = depth_cube.data[:,0,:,:]

(10, 192, 256)
(10, 192, 256)


In [102]:
easy_zonal_mean_cube = raw_data_cube[:,0,:,:].collapsed(['longitude'], iris.analysis.MEAN, weights=easy_depth_array)

In [110]:
print easy_zonal_mean_cube

sea_water_potential_temperature / (K) (time: 10; latitude: 192)
     Dimension coordinates:
          time                             x             -
          latitude                         -             x
     Scalar coordinates:
          depth: 5.0 m, bound=(0.0, 10.0) m
          longitude: 180.0 degrees, bound=(0.0, 360.0) degrees
     Attributes:
          CCCma_data_licence: 1) GRANT OF LICENCE - The Government of Canada (Environment Canada) is...
          CCCma_parent_runid: IGA
          CCCma_runid: IGM
          Conventions: CF-1.5
          associated_files: baseURL: http://cmip-pcmdi.llnl.gov/CMIP5/dataLocation gridspecFile: gridspec_ocean_fx_CanESM2_historical_r0i0p0.nc...
          branch_time: 171915.0
          branch_time_YMDH: 2321:01:01:00
          cmor_version: 2.5.4
          contact: cccma_info@ec.gc.ca
          creation_date: 2011-03-28T14:13:14Z
          drift_removal: Masked 0 of 1966080 points because cubic fit was poor
          experiment: historica

In [109]:
print easy_zonal_mean_cube[0, 50]

sea_water_potential_temperature / (K) (scalar cube)
     Scalar coordinates:
          depth: 5.0 m, bound=(0.0, 10.0) m
          latitude: -42.3291740417 degrees, bound=(-42.7943496704, -41.8639984131) degrees
          longitude: 180.0 degrees, bound=(0.0, 360.0) degrees
          time: 1941-07-02 12:00:00, bound=(1941-01-01 00:00:00, 1942-01-01 00:00:00)
     Attributes:
          CCCma_data_licence: 1) GRANT OF LICENCE - The Government of Canada (Environment Canada) is...
          CCCma_parent_runid: IGA
          CCCma_runid: IGM
          Conventions: CF-1.5
          associated_files: baseURL: http://cmip-pcmdi.llnl.gov/CMIP5/dataLocation gridspecFile: gridspec_ocean_fx_CanESM2_historical_r0i0p0.nc...
          branch_time: 171915.0
          branch_time_YMDH: 2321:01:01:00
          cmor_version: 2.5.4
          contact: cccma_info@ec.gc.ca
          creation_date: 2011-03-28T14:13:14Z
          drift_removal: Masked 0 of 1966080 points because cubic fit was poor
          ex

In [114]:
print easy_zonal_mean_cube.data[0, 50]
print raw_data_cube.data[0,0,50,:].mean()

283.637042548
283.666666667


In [117]:
print depth_cube.data[50,:]

[2809.199951171875 3889.7998046875 4650.599609375 4650.599609375
 4650.599609375 4650.599609375 4650.599609375 4650.599609375 4650.599609375
 4650.599609375 5038.0 5038.0 4650.599609375 5038.0 5038.0 5038.0
 4650.599609375 4267.39990234375 4267.39990234375 4267.39990234375 5038.0
 5038.0 5038.0 5038.0 4650.599609375 4267.39990234375 4267.39990234375
 3889.7998046875 3158.39990234375 2474.7998046875 2474.7998046875
 2809.199951171875 2809.199951171875 3158.39990234375 3158.39990234375
 3158.39990234375 3519.39990234375 4267.39990234375 4650.599609375
 4650.599609375 5038.0 5038.0 5038.0 5038.0 5038.0 4650.599609375
 4267.39990234375 4267.39990234375 4267.39990234375 4267.39990234375
 3889.7998046875 3889.7998046875 3519.39990234375 3158.39990234375
 3158.39990234375 3158.39990234375 3158.39990234375 3158.39990234375
 3158.39990234375 3158.39990234375 3158.39990234375 3158.39990234375
 3158.39990234375 3158.39990234375 3158.39990234375 3158.39990234375
 3158.39990234375 3519.39990234375 

## Case 2: No depth file

This case also applies for curvilinear grids, since calculating the vertical mean field requires regridding to a rectilinear grid first, rendering the depth file useless.

In [119]:
print raw_data_cube[0,...]

sea_water_potential_temperature / (K) (depth: 40; latitude: 192; longitude: 256)
     Dimension coordinates:
          depth                             x             -               -
          latitude                          -             x               -
          longitude                         -             -               x
     Scalar coordinates:
          time: 1941-07-02 12:00:00, bound=(1941-01-01 00:00:00, 1942-01-01 00:00:00)
     Attributes:
          CCCma_data_licence: 1) GRANT OF LICENCE - The Government of Canada (Environment Canada) is...
          CCCma_parent_runid: IGA
          CCCma_runid: IGM
          Conventions: CF-1.5
          associated_files: baseURL: http://cmip-pcmdi.llnl.gov/CMIP5/dataLocation gridspecFile: gridspec_ocean_fx_CanESM2_historical_r0i0p0.nc...
          branch_time: 171915.0
          branch_time_YMDH: 2321:01:01:00
          cmor_version: 2.5.4
          contact: cccma_info@ec.gc.ca
          creation_date: 2011-03-28T14:13:14Z
    

In [125]:
#numpy.nonzero(m.mask)
mask = raw_data_cube[0,...].data.mask
print mask.shape
indices = numpy.nonzero(mask)
print len(indices)

(40, 192, 256)
3


In [132]:
numpy.transpose(indices).shape

(890784, 3)

In [130]:
192 * 256 *40

1966080

In [ ]:
def get_depth(cube, depth_axis):
    